# Loading of the uncapitalized text

This should calculate all the counts.

In [2]:
#libraries
import string
import re
import numpy as np
import math


#load text
f = open("MrZoat_WithThisRingS01_Gould.txt")
text = f.read()
f.close()
#print(text[0:200])

text = text.lower()


# these are most likely not the most effective variants, but it is irrelevant here
sim = ['*', '+', '|', '/', ':', '-', ';', '\'', '\"', ',', '.', '?', '!', '$', '%', '&', '}', '[', ']', '(', ')', '<', '>', ' ', '\n']

low = []
for i in range(26):
    low.append(string.ascii_lowercase[i])
num = []
for i in range(10):
    num.append(string.digits[i])

In [3]:
# now in the correct order
sim1 = ['\n', ' ', '!', '\"', '$', '%', '&', '\'', '(', ')', '*', '+', ',', '-', '.', '/' ]
sim2 = [':', ';', '<', '>', '?', '[', ']']
sim3 = ['|', '}']
ch = sim1 + num + sim2 + low + sim3 # this is our full list of symbols
len(ch)

61

In [4]:
# numpy to make 3-D arrays easily; they are all counts
count3 = np.zeros((61,61,61), int)

str3 = '000'

# again, this is very slow
# I don't think predefining str3 helps
for i in range(61):
    for j in range(61):
        for k in range(61):
            str3 = ch[i]+ch[j]+ch[k]
            count3[i,j,k] = text.count(str3)
            
num_non0 = np.count_nonzero(count3) # much smaller -- 94.4% of cases don't exist
print(num_non0)
c3_non0 = np.nonzero(count3)

12699


In [5]:
# initialize variables
ii = jj = kk = 0
str4 = '0000'
count4 = np.zeros((num_non0, len(ch)), int)

for i in range(num_non0):
    ii=c3_non0[0][i]
    jj=c3_non0[1][i]
    kk=c3_non0[2][i]
    for j in range(len(ch)):
        str4 = ch[ii]+ch[jj]+ch[kk]+ch[j]
        count4[i,j] = text.count(str4)

num4_non0 = np.count_nonzero(count4)
print(num4_non0)
c4_non0 = np.nonzero(count4)

59088


In [6]:
str2 = '00'
count2=np.zeros((61,61), int)

for i in range(61):
    for j in range(61):
        str2 = ch[i]+ch[j]
        count2[i,j] = text.count(str2)

num2_non0 = np.count_nonzero(count2)
print(num2_non0)
c2_non0 = np.nonzero(count2)

1427


In [7]:
# initialize variables
ii = jj = 0
str3 = '000'
count3n = np.zeros((num2_non0, len(ch)), int)

for i in range(num2_non0):
    ii=c2_non0[0][i]
    jj=c2_non0[1][i]
    for j in range(len(ch)):
        str3 = ch[ii]+ch[jj]+ch[j]
        count3n[i,j] = text.count(str3)

num3n_non0 = np.count_nonzero(count3n)
print(num3n_non0)
c3n_non0 = np.nonzero(count3n)

12699


In [8]:
count1=np.zeros((61), int)

for i in range(61):
    count1[i] = text.count(ch[i])

num1_non0 = np.count_nonzero(count1)
print(num1_non0)
c1_non0 = np.nonzero(count1)

61


In [10]:
c4_arr = [0]*num4_non0
for i in range(num4_non0):
    c4_arr[i] = count4[c4_non0[0][i], c4_non0[1][i]]

# Unencoded Count and Numerical Encoding Tests

This is my attempt to use numerical compression for the data, but I found that a fixed width number is shorter.

We can instead ask what it costs to save all the contexts for our 42210 + 10158 + 1258 + 57 + 1 = 53684 potential counts. We can also calculate for 59088 + 12699 + 1427 + 61 + 1 = 73276 when we just want to calculate for non-empty counts. I am not quite sure when we should cut off a context.

Each character can be encoded in 6 bits as we have 61 characters and counts can be given with a number encoding. Characters can be further reduced by examining spacing between characters rather than needing to provide the characters, as the structure is ordered.

 - Provide array of non-zero symbols. (25 bytes typically, although this could be reduced by counting spacing instead.) If we encode each character in 8 bits, we may not need to provide this. We end this with a border (like 'A' or a number which takes the spacing beyond ASCII).
 - Provide counts for context '': Each count is 1 character and one positive integer long. We have 61 counts.
 - Provide counts for context 'x': Each count is 2 characters and one positive integer long. We have 1427 counts.
 - Provide counts for context 'xx': Each count is 3 characters and one positive integer long. We have 12699 counts.
 - Provide counts for context 'xxx': Each count is 4 characters and one positive integer long. 
 - End of each array can be indicated by:
   a) With full character encoding, or 6 bit encoding, the next context will start again at the begining of the alphabet.
   b) With only spacing counted, we can save a mark or just go beyond the end of our character array length.

Each character is 1 byte. Our maximum count is 724362, which requires 20 bits to encode. We could do this with 3 bytes (but not 2 bytes), but 4 bytes is more typical. Total cost, with typical encoding:
5* 61 + 6 * 1427 + 7 * 12699 + 8 * 59088 = 570464 bytes = 0.57 MB.

Spacings: (the last 3 is for the end of the list)
This assumes all digits and letters are used.
 - p1: {10, 22, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2, 5, 1, 3} -- 3 at the end is used to indicate end of file
 - p2: {count1} -- without any charater spacing, because all are used
 - p3: {c2_non0\[1\]\[0\]+1, c2_non0\[1\]\[1:\]- c2_non0\[1\]\[0:-1\]} {count2} -- The first character spacing is removed, since all are going to exist. The second is as given, but the negative values in it need to be recalculated (+61 I think, I need to check this). This will indicate incrementing the first character by 1. Going to 61 is the most effecient version, and works. The additional +1 is already included.
 - x = 60 - the last index -- this will encode the first non-existing count, which indicates end of list
 - p4: {c3_non0\[1\]\[0\]+1, c3_non0\[1\]\[1:\]- c3_non0\[1\]\[0:-1\]} {c3_non0\[2\]\[0\]+1, c3_non0\[2\]\[1:\]- c3_non0\[2\]\[0:-1\]} {count3} -- Rules like the previous.
 - y = 60 - the last index -- as x
 - p5: {c4_non0\[1\]\[0\]+1, c4_non0\[1\]\[1:\]- c4_non0\[1\]\[0:-1\]} {c4_non0\[2\]\[0\]+1, c4_non0\[2\]\[1:\]- c4_non0\[2\]\[0:-1\]} {c4_non0\[3\]\[0\]+1, c4_non0\[3\]\[1:\]- c4_non0\[3\]\[0:-1\]} {count4} -- For this case, the formalism is not actually correct for how I have calculated the data, which uses c3 numbers for calculating c4 numbers. However, the structure for transmition is similar
 -  z = 60 - the last index, to indicate end of file, may now add the text

All numbers will be encoded with ternary code, to maintain consistency, despite not being the best choise. I should calculate costs, separating spacings and counts as two separate encodings. Counts most likely want ternary code, while this is likely inefficent for spacings.

9 bits is the greatest spacing. However, this encoding doesn't quite work. The alternative encoding, which I did for c4, does work. 
 - p1: {10, 22, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2, 5, 1, 3}
 - p2: {count1}
 - p3: {c2_non0\[1\]\[0\]+1, c2_non0\[1\]\[1:\]- c2_non0\[1\]\[0:-1\]} {count2}
 - x = 60 - the last index
 - p4: {c3n_non0\[1\]\[0\]+1, c3n_non0\[1\]\[1:\]- c3n_non0\[1\]\[0:-1\]} {count3n} -- Like p2, we have a context (the set of all non-zero contexts from p3), which always increments by 1, and our added letter counts. Therefore, everytime there will be exactly 2 numbers, a spacing and a count. We know which previous contexts exist from the previous process. We deal with negatives in our array like indicated before.
 - y = 60 - the last index
 - p5: {c4_non0\[1\]\[0\]+1, c4_non0\[1\]\[1:\]- c4_non0\[1\]\[0:-1\]} {count4} -- Like previous. This is the correct formulation of my calculation of counts.
 - z = 60 - the last index, to indicate end of file

Except for the first few characters, every character has a 3 character preceding context. However, I wish to keep all the single count elements and the smaller machines due to how they count the context. I can recalculate the contexts as below:

 - p1: {10, 22, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2, 5, 1, 3}
 - p2: {'ABC'} -- The first three characters of the document, encoded from 1 - 61 (index 0 - 60). 
 - p3: {c2_non0\[1\]\[0\]+1, c2_non0\[1\]\[1:\]- c2_non0\[1\]\[0:-1\]} -- Add 61 to all non-positive.
 - x = 60 - the last index (note, last index here, not last spacing)
 - p4: {c3n_non0\[1\]\[0\]+1, c3n_non0\[1\]\[1:\]- c3n_non0\[1\]\[0:-1\]} -- Add 61 to all non-positive.
 - y = 60 - the last index (note, last index here, not last spacing)
 - p5: {c4_non0\[1\]\[0\]+1, c4_non0\[1\]\[1:\]- c4_non0\[1\]\[0:-1\]} {count4} -- Add 61 to all non-positive character spacings. Counts are only important here, as the old counts can be calculated from these counts and the first 3 characters.
 - z = 60 - the last index, to indicate end of file (note, last index here, not last spacing)

I should be able to here calculate the bit / byte length and figure out which number encodings are better. 

***

I have a problem with the encoding -- if the index of the last element of the old context is less than the first element of the next, the encoder / decoder will not recognize the requirement to increment to the next element. I can fix this by inserting an unused number (62 or 63) between the two elements, to indicate the need to increment. I have not yet added this fix to the algorithm (due to time constraints). The increase in size should be relatively small. 

Also, I can just put the indicies in straight, and not use the spacings. This doesn't alter the space usage, but decreases processing time. 

The first array needs to have a separate encoding (and be based on spacing) as we don't know the size of our array until after reading the first array. (bit size -> 7, 6, 6, 6, {5}, 5, ..., 5, {4}, 4, 4, 4, 4, {2}, 2 or just all 7) Now we can calculate the size of the array = 26 + 10 + number of special elements. We have 1 more element as our stop, to indicate needing to increment the first index by one if it is not obvious. When this index reaches the size of our previous array, we know that the next number will be for the next array. The last byte is finished with zeros (they will be ignored).

All future calculation of efficency is not effected by this change. The size will increase, but not significantly. 

Our count number also needs something to indicate the size of the array. After the p5 array, before the extra 0s and count array, I should add a ternary encoded number for the number of bits in the maximum count, defining the size of the next array. I don't want a fixed-length encoding here, in order to not assume the size of the maximum count. Then I should ignore the trailing 0s until the end and load the next numbers straight onto the byte string, as there is no reason to believe they will be a whole number of bits (although they are).

Unencoded 6 bit length for the spacings is optimal, it looks like. I can just add zeros to the end to reach a full byte if needed due to the fact that I already signalled the end of file. It is likely unneeded.

Our total cost after encoding is 173 kB, which is less than half the old calculation. Note that, despite the unencoded form, there are still two separate types of encoding due to the difference in bit length of the numbers. I wonder if I can just label them as uint * 6 and uint * 16 and just write the binary. There is no issue with putting the counts at the end, as we already know the order and number of counts, so we know what to expect. We need 4 extra zeros to reach a whole number of bits, but they still can be added to the end of the file.

170 kB, success, although I need to read it properly to truely check. Also .txt is not correct.

# Classes

 - byte_mng()
 - count_mng(alphabet_in=None, counts_in=None, new_mode=2)
 - arith_code_mng(counts_mngr, bits_store) -- requires an instance of both of the above classes

In [11]:
# - store bits and bytes, take bits and add it to the byte list (input array of bools, update encoding)
class byte_mng:
    def __init__(self):
        self.byte_store = [0] # here I am filling the bytes, should be integers from 0 to 255
        self.bit_count = 0    # this is the number of bits in byte_store which are full; ensures that I don't overfill bytes and send them to the string when ready
        self.j = 0            # counts the length of byte_store
        
    def load_bits(self, bits, lngth):
        # the old bits are already shifted appropriately
        jj = self.byte_store[self.j]
        
        # bit shift new bits
        if self.bit_count < (8 - lngth): 
            ii = bits << (8 - lngth - self.bit_count)
        elif self.bit_count == (8 - lngth):
            ii = bits
        else:
            ii = bits >> (self.bit_count - 8 + lngth) # The excess will go off the end. I will encode them later.

        #print(lngth, self.bit_count, jj, ii)
        # bitwise or is the same as + in this case
        self.byte_store[self.j] = jj | ii
        used_bits = 8 - self.bit_count     # for when bit_count + lngth > 7
        self.bit_count += lngth

        bits_n = bits
        lngth_n = lngth

        #print(lngth, self.bit_count, jj, ii)
        # now, to work on the next byte
        while self.bit_count > 7:
            #print('here')
            self.j += 1
            self.bit_count -=8
            
            # we now need to take only the right most bit_count bits and put it in ii
            lngth_n -= used_bits
            bits_n = bits_n % (2**lngth_n) # removes the used bits
            if self.bit_count < 8:
                ii = (bits_n << 8 - lngth_n)%256
            else:
                ii = bits_n >> (lngth_n - 8)
                used_bits = 8
                
            self.byte_store.append(ii)

    # now for the code for ternary code
    def load_ternary(self, num):
        # 00, 01, 10, 11
        # leading will be 1 or 0

        # number of digits for base 3 representation
        n = 0
        x = num
        while x >= 1:
            x = x / 3
            n += 1
        n2 = n

        # start with the leading bit
        # there are two variants only: 1 or 2
        n = n-1
        x = num // 3**n
        y = num % 3**n

        #print(x,y, x-1)
        self.load_bits(x-1,1) # 1 for 2 and 0 for 1
        
        # repeat for the rest of the digits
        for i in range(n2-1):
            n = n-1
            x = y // 3**n
            y = y % 3**n
            #print(x,y,1+x)
            self.load_bits(1+x,2) # 01 = 0, 10 = 1, 11 = 2

        # and load the end of number code
        self.load_bits(0,2)
            
    def save_bits(self, filename):
        self.byte_encode = bytearray(self.byte_store)
        with open(filename, "wb") as f:
            f.write(self.byte_encode)

In [194]:
class count_mng:
    # defines three modes 
    # 0 means static counts,
    # 1 means dynamic, fixed alphabet
    # 2 means dynamic alphabet
    # new_mode defines the count of the letter 'new' in the dynamic alphabet
    def __init__(self,alphabet_in=None, counts_in=None, new_mode=2):
        self.mode = 0
        if alphabet_in == None:
            self.mode = 2
            self.alphabet = []
            self.counts = []
        elif counts_in == None:
            self.mode = 1
            self.alphabet = alphabet_in
            self.counts = [1]*len(alphabet_in)
        else:
            self.mode = 0
            self.alphabet = alphabet_in
            self.counts = counts_in
        self.new_mode = new_mode

    def get_counts(self, char):
        num_char = self.find_num_char(char)
        if self.mode == 0: return self.get_counts_0(num_char)
        elif self.mode == 1: return self.get_counts_1(num_char)
        else: return self.get_counts_2(num_char, char)

    def get_counts_0(self,num_char):
        return [sum(self.counts[:num_char]), self.counts[num_char], sum(self.counts[num_char+1:])]
    
    def get_counts_1(self,num_char):
        countret = [sum(self.counts[:num_char]), self.counts[num_char], sum(self.counts[num_char+1:])]
        self.counts[num_char] += 1
        return [countret[0], countret[1], countret[2]]
    
    # this is the most complicated case
    # I need to find the structure here and add it
    def get_counts_2(self,num_char, char):
        if num_char < len(self.alphabet):
            if self.counts[num_char] == 0:
                countret = [sum(self.counts), self.get_count_new(), 0]
            else:
                countret = [sum(self.counts[:num_char]), self.counts[num_char], sum(self.counts[num_char+1:])+self.get_count_new()]
            self.counts[num_char] += 1
            return [countret[0], countret[1], countret[2]]
        else:
            countret = [sum(self.counts), self.get_count_new()]
            self.add_new_count(char)
            return [countret[0], countret[1], 0]
        
    # both of the following are for dynamic alphabets
    def get_count_new(self):
        modes = [1, 
                 max(1,len(self.alphabet)), 
                 max(1,len(self.alphabet)), 
                 max(1.0, len(self.alphabet)/2)]
        return modes[self.new_mode]
    
    def add_new_count(self, char):
        self.alphabet.append(char)
        modes = [1,
                 0,
                 1,
                 0.5]
        self.counts.append(modes[self.new_mode])

    # this will need to return the corresponding index in the alphabet for the number
    # in the case of mode == 2, I need to return len(alphabet) for char not in alphabet
    def find_num_char(self, char):
        if self.mode == 2 and char not in self.alphabet:
            return len(self.alphabet)
        else:
            return self.alphabet.index(char)

In [178]:
# - keep track of and update l, h, bits
# {l_old, h_old, bits_old -> a, b -> h-l+1, h-l+1 / 2^n -> a+ term, b+ term -> l, 
#  h -> bin l, bin h -> enc update -> new bin l, new bin h, bits_new -> new l, new h}
class arith_code_mng:
    def __init__(self, counts_mngr, bits_store):
        # window size
        self.N = 40
        self.win_size = 2**self.N

        # left and right borders
        # I know I don't need so many, but it should be fine right now
        self.l = 0
        self.h = self.win_size - 1
        self.bits = 0
        self.lo = 0
        self.ho = self.win_size - 1
        self.ln = 0
        self.hn = self.win_size - 1

        #counts
        self.find_counts = counts_mngr

        # encoding
        self.byte_encode = bits_store

        self.duration_static = 0

    def find_probs(self, counts):
        den = float(counts[0]+counts[1]+counts[2])
        num1 = float(counts[0])
        num2 = float(counts[0]+counts[1])
        lb_fl = num1 / den
        rb_fl = num2 / den
        lb = math.ceil(lb_fl * self.win_size)
        rb = math.ceil(rb_fl * self.win_size) - 1
        return lb, rb

    def add_character(self, char):
        counts = self.find_counts.get_counts(char)
        a,b = self.find_probs(counts)

        #print("ch: ", char, "a,b: ", format(a, '032b'), format(b, '032b'))
        
        hl1_N = float(self.ho-self.lo+1) / float(self.win_size)
        a_pl = math.ceil(hl1_N * a)
        b_pl = math.ceil(hl1_N * (b + 1)) - 1

        #print(format(self.window, '032b'))

        self.l = self.lo + a_pl
        self.h = self.lo + b_pl

        #print("ch: ", char, "l,h:", format(self.l, '032b'), format(self.h, '032b'))

        #print("new l, h: ", float(self.h-self.l))
        enc_n, bit_n = self.encode() # loads bits into the bit array and records how many bits need to be removed from l and h
        self.new_lh(enc_n, bit_n) # removes bits from l and h
        #print("remove: ", float(self.hn-self.ln), enc_n, bit_n)

        # updates for the next run
        self.lo = self.ln
        self.ho = self.hn

    # This appears to work properly.
    # This loads the bits into the array and updates bits, but so far doesn't update l and h
    def encode(self):
        l = self.l
        h = self.h

        nbitload = 0
        nbits = 0

        #print(self.h-self.l)
        # both l and h are N bits long
        x = self.win_size
        #print(l / x, h / x)

        lb = 0
        hb = 0
        
        for i in range(self.N):
            #print("bits:", i, lb,hb, l/x, h/x)
            x = x // 2
            lb = l // x
            l = l % x
            hb = h // x
            h = h % x
            #print("bits:", i, lb,hb, l/x, h/x, self.bits)
            if lb == hb:
                self.byte_encode.load_bits(lb, 1)
                while self.bits > 0:
                    self.byte_encode.load_bits(not(lb),1)
                    self.bits -= 1
                nbitload += 1
            else:
                #print(i)
                break

        # the final 0s in the file
        if nbitload == 0:
            self.duration_static += 1
        else:
            self.duration_static = 0

        # widening the window
        # print(l / x, h / x)
        for i in range(self.N - nbitload - 1):
            x = x // 2
            lb = l // x
            l = l % x
            hb = h // x
            h = h % x
            if lb == 1 and hb == 0:
                nbits += 1
                self.bits += 1
            else:
                break

        #print(self.h-self.l, nbitload, nbits)
        return nbitload,nbits

    # again, I need to check this and check the logic
    def new_lh(self, nbitload, nbits):
        # removing bits
        self.ln = (self.l << nbitload) % self.win_size
        self.hn = (self.h << nbitload) % self.win_size
        self.hn = self.hn | (2**nbitload - 1) # fill the end with 1s, not 0s

        # widening of the window
        if nbits > 0:
            l_fb = (self.ln // 2**(self.N-1)) << (self.N-1)
            h_fb = (self.hn // 2**(self.N-1)) << (self.N-1)
            self.ln = (self.ln % 2**(self.N-1-nbits)) << nbits
            self.hn = (self.hn % 2**(self.N-1-nbits)) << nbits
            self.hn = self.hn | (2**nbits - 1) # fill the end with 1s, not 0s
            # and return the first bit to the array
            self.ln = self.ln | l_fb
            self.hn = self.hn | h_fb

    # I need error-checking here to prevent using any routines if this has already been applied
    def finalize(self):
        self.byte_encode.load_bits(1,1) # this is always 1, because 00, 01, 11 are the only possibilities, and 00 and 11 will already be removed.
        if self.duration_static > 0:
            self.byte_encode.load_bits(0, self.duration_static) # how many final characters the fraction was unchanged
            
        # now to finalize the bytestream
        a = 8 - self.byte_encode.bit_count
        # the encoding is already correct unless a = 8
        if a == 8:
            a = 0
            # remove the last zero byte
            b = self.byte_encode.byte_store.pop()
        return a
    
    # I am pretty sure that there is a more intelligent way to do this. But this is easiest and only 4 bytes (total, because called only once).
    def partial_finalize(self):
        self.byte_encode.load_bits(1,1) # this is always 1, because 00, 01, 11 are the only possibilities, and 00 and 11 will already be removed.
        self.byte_encode.load_bits(0, self.N-1) # to the end of the window

# Final Numerical Encoding Variant

This is my final encoding with fixed-length numerical encoding, removal of repeated information, and no other data compression. This encoding works for decoding the text, and has a size of 170 kB.

New variant:

 - p1: {10, 22, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2, 5, 1, 3}
 - size: (bit size -> 7, 6, 6, 6, {5}, 5, ..., 5, {4}, 4, 4, 4, 4, {2}, 2 or just all 7)
 - p2: {'ABC'} -- The first three characters of the document (index 0 - 60). 
 - p3: {c2_non0\[1\]\[0\]+1, c2_non0\[1\]\[1:\]- c2_non0\[1\]\[0:-1\]} -- Add 61 to all non-positive.
 - x = 61
 - p4: {c3n_non0\[1\]\[0\]+1, c3n_non0\[1\]\[1:\]- c3n_non0\[1\]\[0:-1\]} -- Add 61 to all non-positive.
 - y = 61
 - p5: {c4_non0\[1\]\[0\]+1, c4_non0\[1\]\[1:\]- c4_non0\[1\]\[0:-1\]} -- Add 61 to all non-positive character spacings. Counts are only important here, as the old counts can be calculated from these counts and the first 3 characters.
 - z = 61, to indicate end of file 
 - w = (bin(max(count4)) - 2) = number of bits in max count - ternary encoding
 - count4

Indicies encoded 0-60 (straight) 61 indicates need to increment the first index only if not obvious.

In [128]:
def np2D_to_list1D(arr, mng_arr, m_ind = 0):
    out_arr = [arr[0]]
    max_ind = max(max(arr) + 1, m_ind) # so that giving this value is also allowed

    for i in range(1,len(arr)):
        # this is my exception, when the increase is not obvious
        # first condition -- that the old array increments, 
        # second condition, that the new index is greater than the previous, which won't increment the first normally
        if (mng_arr[i] != mng_arr[i-1]) and (arr[i] > arr[i-1]):
            out_arr.append(max_ind)
        out_arr.append(arr[i])

        # small bit of error-checking
        if (mng_arr[i] != mng_arr[i-1]) and (mng_arr[i] != (mng_arr[i-1] + 1)):
            print('error: ', i, ', ', mng_arr[i], ', ', mng_arr[i-1])

    return out_arr

We will now load the alphabet into the byte stream. The encoding is simple, with a known byte width based on the remaining number of potential characters. It decodes properly. This leads us into arithmetic encoding with a known alphabet and a byte stream which has already been initialized.

In [232]:
counts_enc = byte_mng()

# p1
# 0 means ending the array
sim_arr = [10, 22, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2, 5, 1, 0]
# - size: (bit size -> 7, 6, 6, 6, {5}, 5, ..., 5, {4}, 4, 4, 4, 4, {2}, 2 or just all 7)
# should be calculated automatically:  calc_sim_size(sim)
size_sim_arr = [7, 6, 6, 6, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 4, 4, 4, 4, 2, 2]

for i in range(len(sim_arr)):
    counts_enc.load_bits(sim_arr[i], size_sim_arr[i])

In [233]:
print(counts_enc.byte_store[-1], counts_enc.bit_count) # also fine; but need to add functionality in future

64 4


# Arithmetic Encoding of Machines

p2 to p5 have a fixed alphabet encription of length 62 (0 to 61). The removal from arithmetic encoding is identical to the case without any encription. 4 bytes are "removed" from the byte stream while decrypting, but can be returned after the end of the file is reached. This means that I can just load charaters into an encoder. End of text is noted like in the unencrypted case, by measuring known byte lengths. I can add 0s to the end, but it shouldn't even be necessary, as excess bits on the end of any type don't change the decoding, and I know they are for the next encoding.

2) p2: {'ABC'} -- The first three characters of the document (index 0 - 60). 
3) p3_arr = c2_non0[1][:], add 61 for unclear boundaries
4) p4_arr = c3n_non0[1][:], add 61 for unclear boundaries
5) p5_arr = c4_non0[1][:], add 61 for unclear boundaries

In [234]:
# set up the alphabet
array_nums = [i for i in range(len(ch)+1)]
counts_dynamic_static = count_mng(alphabet_in=array_nums)
enc_mng_ds = arith_code_mng(counts_mngr=counts_dynamic_static, bits_store=counts_enc)

In [ ]:
#p2
f3_num = [ch.index(text[0]), ch.index(text[1]), ch.index(text[2])]
for i in f3_num:
    enc_mng_ds.add_character(i)

print('p2: ', len(f3_num),counts_enc.bit_count, len(counts_enc.byte_store)+4)

p2:  3 5 23


In [236]:
#p3
p3_arr = np2D_to_list1D(c2_non0[1][:], c2_non0[0][:], 61)
for i in p3_arr:
    enc_mng_ds.add_character(i)
enc_mng_ds.add_character(61)

print('p3: ', len(p3_arr), counts_enc.bit_count, len(counts_enc.byte_store)+4)

p3:  1429 7 1051


In [237]:
#p4
p4_arr = np2D_to_list1D(c3n_non0[1][:], c3n_non0[0][:], 61)
for i in p4_arr:
    enc_mng_ds.add_character(i)
enc_mng_ds.add_character(61)

print('p4: ', len(p4_arr), counts_enc.bit_count, len(counts_enc.byte_store))

p4:  12848 3 9583


In [238]:
#p5
p5_arr = np2D_to_list1D(c4_non0[1][:], c4_non0[0][:], 61)
for i in p5_arr:
    enc_mng_ds.add_character(i)
enc_mng_ds.add_character(61)

print('p5: ', len(p5_arr), counts_enc.bit_count, len(counts_enc.byte_store))

p5:  61366 6 48640


In [239]:
# end of first arithmetic encoding
enc_mng_ds.partial_finalize()

Now we go to the counts. We know how many characters (numbers). We no longer need the max length as each count is encoded as a seperate character.

In [240]:
counts_dynamic = count_mng()
enc_mng_d = arith_code_mng(counts_mngr=counts_dynamic, bits_store=counts_enc)

In [241]:
for i in c4_arr:
    enc_mng_d.add_character(i)

print('p6: ', len(c4_arr),counts_enc.bit_count, len(counts_enc.byte_store))

p6:  59088 3 88076


In [243]:
enc_mng_d.partial_finalize()

In [ ]:
for i in range(len(counts_dynamic.alphabet)):
    counts_enc.load_ternary(counts_dynamic.alphabet[i])
    print(counts_dynamic.alphabet[i], counts_enc.bit_count, len(counts_enc.byte_store), counts_enc.byte_store[-1])

In [177]:
print(counts_dynamic_static.counts)
print(len(counts_dynamic.alphabet), counts_dynamic.alphabet)
print(counts_dynamic.counts)

[1406, 5014, 1068, 1283, 4, 11, 12, 1590, 16, 87, 779, 7, 1781, 1701, 2876, 106, 171, 180, 175, 131, 116, 110, 119, 86, 71, 78, 318, 285, 4, 7, 1408, 32, 45, 4026, 1400, 1835, 1781, 4110, 1335, 1555, 2112, 3817, 463, 1041, 2700, 1855, 2616, 3519, 1483, 421, 2961, 3174, 2764, 2530, 864, 1370, 296, 1743, 422, 4, 4, 2433]
1476 [4, 1, 3, 8, 5, 6, 18, 24, 17, 7, 2, 32, 98, 71, 1066, 336, 276, 441, 151, 180, 221, 791, 2179, 78, 134, 169, 371, 876, 706, 281, 443, 718, 1316, 234, 30, 1305, 1213, 1072, 25, 49, 10, 40, 12, 9, 11, 13, 14, 16, 248, 23, 19, 54, 95, 279, 34, 178, 20, 184, 26, 51, 48, 22, 147, 45, 84, 82, 15, 35, 42, 41, 57, 1033, 160, 46, 44, 43, 2247, 224, 31, 53, 29, 105, 27, 201, 213, 185, 159, 106, 263, 165, 64, 37, 36, 68, 368, 156, 775, 58, 59, 87, 1186, 52, 290, 269, 188, 86, 141, 124, 326, 128, 33, 96, 99, 21, 116, 245, 62, 102, 350, 39, 192, 1017, 396, 65, 166, 107, 327, 92, 776, 73, 297, 55, 80, 61, 176, 187, 115, 76, 278, 56, 1160, 38, 114, 100, 393, 679, 85, 536, 647, 60

In [191]:
#whitespace2 = enc_mng_d.finalize() # remove the trailing null byte if it exists
counts_enc.save_bits('count_enc_test')

In [246]:
len(counts_enc.byte_store)

90640

size is now 89 kB